Setup and Imports

In [14]:
from web3 import Web3
from eth_account import Account
import json
import os
from dotenv import load_dotenv
load_dotenv()

# Connect to network
w3 = Web3(Web3.HTTPProvider('https://api.hyperliquid-testnet.xyz/evm'))

# Setup accounts
deployer_key = os.getenv('DEPLOYER_PRIVATE_KEY')
staker_key = os.getenv('STAKER_PRIVATE_KEY')
deployer_account = Account.from_key(deployer_key)
staker_account = Account.from_key(staker_key)

Deploy Staking Contract

In [15]:
# Load contract ABIs and bytecode
with open('./bin/contracts/MTKStaking.abi', 'r') as f:
    staking_abi = json.loads(f.read())
with open('./bin/contracts/MTKStaking.bin', 'r') as f:
    staking_bytecode = f.read()

# Standard ERC20 ABI
token_abi = [
    {
        "constant": True,
        "inputs": [],
        "name": "name",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [{"name": "_spender", "type": "address"}, {"name": "_value", "type": "uint256"}],
        "name": "approve",
        "outputs": [{"name": "", "type": "bool"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "totalSupply",
        "outputs": [{"name": "", "type": "uint256"}],
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [{"name": "_from", "type": "address"}, {"name": "_to", "type": "address"}, {"name": "_value", "type": "uint256"}],
        "name": "transferFrom",
        "outputs": [{"name": "", "type": "bool"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [{"name": "_to", "type": "address"}, {"name": "_value", "type": "uint256"}],
        "name": "transfer",
        "outputs": [{"name": "", "type": "bool"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}, {"name": "_spender", "type": "address"}],
        "name": "allowance",
        "outputs": [{"name": "", "type": "uint256"}],
        "type": "function"
    }
]

# Get token address
token_address = Web3.to_checksum_address(os.getenv('TOKEN_ADDRESS'))

# Create contract factory
MTKStaking = w3.eth.contract(abi=staking_abi, bytecode=staking_bytecode)

# Build deployment transaction
deploy_transaction = MTKStaking.constructor(token_address).build_transaction({
    'from': deployer_account.address,
    'nonce': w3.eth.get_transaction_count(deployer_account.address),
    'gas': 3000000,  # Increased gas limit for contract deployment
    'gasPrice': w3.eth.gas_price
})

# Sign and send deployment transaction
signed_txn = w3.eth.account.sign_transaction(deploy_transaction, deployer_key)
tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

staking_address = tx_receipt.contractAddress
print(f"Staking contract deployed at: {staking_address}")

# Create contract instances
token = w3.eth.contract(address=token_address, abi=token_abi)
staking = w3.eth.contract(address=staking_address, abi=staking_abi)

Staking contract deployed at: 0x34C97B97eeA68BB84C351C645A185fc2A8548C96


Test the staking contract

In [16]:
# Load contract ABIs
with open('./bin/contracts/MTKStaking.abi', 'r') as f:
    staking_abi = json.loads(f.read())
    
# Load standard token abi
token_abi = [
    {
        "constant": True,
        "inputs": [],
        "name": "name",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [{"name": "_spender", "type": "address"}, {"name": "_value", "type": "uint256"}],
        "name": "approve",
        "outputs": [{"name": "", "type": "bool"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "totalSupply",
        "outputs": [{"name": "", "type": "uint256"}],
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [{"name": "_from", "type": "address"}, {"name": "_to", "type": "address"}, {"name": "_value", "type": "uint256"}],
        "name": "transferFrom",
        "outputs": [{"name": "", "type": "bool"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function"
    },
    {
        "constant": False,
        "inputs": [{"name": "_to", "type": "address"}, {"name": "_value", "type": "uint256"}],
        "name": "transfer",
        "outputs": [{"name": "", "type": "bool"}],
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}, {"name": "_spender", "type": "address"}],
        "name": "allowance",
        "outputs": [{"name": "", "type": "uint256"}],
        "type": "function"
    },
    {
        "anonymous": False,
        "inputs": [{"indexed": True, "name": "owner", "type": "address"}, {"indexed": True, "name": "spender", "type": "address"}, {"indexed": False, "name": "value", "type": "uint256"}],
        "name": "Approval",
        "type": "event"
    },
    {
        "anonymous": False,
        "inputs": [{"indexed": True, "name": "from", "type": "address"}, {"indexed": True, "name": "to", "type": "address"}, {"indexed": False, "name": "value", "type": "uint256"}],
        "name": "Transfer",
        "type": "event"
    }
]


# Contract addresses
token_address = os.getenv('TOKEN_ADDRESS')
staking_address = os.getenv('STAKING_ADDRESS')

# Create contract instances
token = w3.eth.contract(address=token_address, abi=token_abi)
staking = w3.eth.contract(address=staking_address, abi=staking_abi)

Check initial balances

In [18]:
# Helper Functions
def print_balances(account_address, label=""):
    token_balance = w3.from_wei(token.functions.balanceOf(account_address).call(), 'ether')
    stake_info = staking.functions.stakes(account_address).call()
    staked_balance = w3.from_wei(stake_info[0], 'ether')  # stake_info[0] is the amount
    print(f"\n{label} Balances:")
    print(f"Token balance: {token_balance} MTK")
    print(f"Staked balance: {staked_balance} MTK")


# Check initial balances
print_balances(deployer_account.address, "Deployer")
print_balances(staker_account.address, "Staker")



Deployer Balances:
Token balance: 997600 MTK
Staked balance: 0 MTK

Staker Balances:
Token balance: 1102.74 MTK
Staked balance: 0 MTK


Deposit with a smaller amount

In [19]:
def check_staking_contract_state():
    try:
        # Check reward pool in contract
        reward_pool = staking.functions.rewardPool().call()
        print(f"Reward pool in contract: {w3.from_wei(reward_pool, 'ether')} MTK")
        
        # Check reward rate
        reward_rate = staking.functions.rewardRate().call()
        print(f"Reward rate: {reward_rate}")
        
        # Check contract token balance
        contract_balance = token.functions.balanceOf(staking_address).call()
        print(f"Contract token balance: {w3.from_wei(contract_balance, 'ether')} MTK")
        
        # Check allowance
        allowance = token.functions.allowance(deployer_account.address, staking_address).call()
        print(f"Contract allowance from deployer: {w3.from_wei(allowance, 'ether')} MTK")
        
    except Exception as e:
        print(f"Error checking contract state: {str(e)}")

def send_transaction(transaction, private_key):
    try:
        # Estimate gas first
        gas_estimate = w3.eth.estimate_gas(transaction)
        transaction['gas'] = int(gas_estimate * 1.2)  # Add 20% buffer
        
        signed_txn = w3.eth.account.sign_transaction(transaction, private_key)
        tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        
        if receipt.status == 0:
            try:
                # Get detailed error
                tx = w3.eth.get_transaction(tx_hash)
                error = w3.eth.call(
                    {
                        'to': tx['to'],
                        'from': tx['from'],
                        'data': tx['input'],
                        'value': tx['value'],
                        'gas': tx['gas'],
                        'gasPrice': tx['gasPrice']
                    },
                    receipt.blockNumber - 1
                )
                print(f"Transaction reverted: {error.hex()}")
            except Exception as call_e:
                print(f"Transaction failed with error: {str(call_e)}")
            return None
        return receipt
    except Exception as e:
        print(f"Transaction failed: {str(e)}")
        return None

def deposit_rewards(amount_in_ether):
    if amount_in_ether <= 0:
        print("Amount must be greater than 0")
        return
        
    try:
        amount = w3.to_wei(amount_in_ether, 'ether')
        
        print("\nBefore deposit:")
        print_balances(deployer_account.address, "Deployer")
        check_staking_contract_state()
            
        # First approve with dynamic gas estimation
        approve_tx = token.functions.approve(
            staking_address,
            amount
        ).build_transaction({
            'from': deployer_account.address,
            'nonce': w3.eth.get_transaction_count(deployer_account.address),
            'gasPrice': w3.eth.gas_price
        })
        
        receipt = send_transaction(approve_tx, deployer_key)
        if not receipt:
            return
        print("\nAfter approval:")
        check_staking_contract_state()

        # Then deposit with dynamic gas estimation
        deposit_tx = staking.functions.depositRewards(
            amount
        ).build_transaction({
            'from': deployer_account.address,
            'nonce': w3.eth.get_transaction_count(deployer_account.address),
            'gasPrice': w3.eth.gas_price
        })
        
        receipt = send_transaction(deposit_tx, deployer_key)
        if not receipt:
            return
            
        print("\nAfter deposit:")
        print_balances(deployer_account.address, "Deployer")
        check_staking_contract_state()
        
        if receipt.status == 1:
            print(f"Successfully deposited {amount_in_ether} MTK as rewards")
            # Check events
            deposit_events = staking.events.RewardsDeposited().process_receipt(receipt)
            if deposit_events:
                print("RewardsDeposited event confirmed")
        else:
            print("Transaction failed")
            
    except Exception as e:
        print(f"Error in deposit_rewards: {str(e)}")

# Try depositing with a smaller amount first
deposit_rewards(100)


Before deposit:

Deployer Balances:
Token balance: 997600 MTK
Staked balance: 0 MTK
Reward pool in contract: 1197.26 MTK
Reward rate: 100000000000000
Contract token balance: 1197.26 MTK
Contract allowance from deployer: 0 MTK

After approval:
Reward pool in contract: 1197.26 MTK
Reward rate: 100000000000000
Contract token balance: 1197.26 MTK
Contract allowance from deployer: 100 MTK

After deposit:

Deployer Balances:
Token balance: 997500 MTK
Staked balance: 0 MTK
Reward pool in contract: 1297.26 MTK
Reward rate: 100000000000000
Contract token balance: 1297.26 MTK
Contract allowance from deployer: 0 MTK
Successfully deposited 100 MTK as rewards
RewardsDeposited event confirmed


/Users/qw246/miniforge3/envs/quant/lib/python3.10/site-packages/eth_utils/functional.py:47: UserWarning: The log with transaction hash: HexBytes('0xfb7dc4fe771c61ef03863aa30fd7c3288488f7097cb29e0b98e0e8c29de86622') and logIndex: 0 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  return callback(fn(*args, **kwargs))


Test staking, claiming, and unstaking

In [20]:
def stake_tokens(amount_in_ether):
    if amount_in_ether <= 0:
        print("Amount must be greater than 0")
        return
        
    try:
        amount = w3.to_wei(amount_in_ether, 'ether')
        
        print("\nBefore staking:")
        print_balances(staker_account.address, "Staker")
        check_staking_contract_state()
            
        # First approve staking contract to spend tokens
        approve_tx = token.functions.approve(
            staking_address,
            amount
        ).build_transaction({
            'from': staker_account.address,
            'nonce': w3.eth.get_transaction_count(staker_account.address),
            'gasPrice': w3.eth.gas_price
        })
        
        # Send approval transaction
        receipt = send_transaction(approve_tx, staker_key)
        if not receipt:
            return
        
        # Then stake tokens
        stake_tx = staking.functions.stake(
            amount
        ).build_transaction({
            'from': staker_account.address,
            'nonce': w3.eth.get_transaction_count(staker_account.address),
            'gasPrice': w3.eth.gas_price
        })
        
        receipt = send_transaction(stake_tx, staker_key)
        if not receipt:
            return
            
        print("\nAfter staking:")
        print_balances(staker_account.address, "Staker")
        check_staking_contract_state()
        
        if receipt.status == 1:
            print(f"Successfully staked {amount_in_ether} MTK")
            # Check events if needed
            stake_events = staking.events.Staked().process_receipt(receipt)
            if stake_events:
                print("Staked event confirmed")
        else:
            print("Staking transaction failed")
            
    except Exception as e:
        print(f"Error in stake_tokens: {str(e)}")

def claim_rewards():
    try:
        print("\nBefore claiming rewards:")
        print_balances(staker_account.address, "Staker")
        
        # Prepare claim transaction
        claim_tx = staking.functions.claimRewards().build_transaction({
            'from': staker_account.address,
            'nonce': w3.eth.get_transaction_count(staker_account.address),
            'gasPrice': w3.eth.gas_price
        })
        
        # Send claim transaction
        receipt = send_transaction(claim_tx, staker_key)
        if not receipt:
            return
        
        print("\nAfter claiming rewards:")
        print_balances(staker_account.address, "Staker")
        
        if receipt.status == 1:
            print("Successfully claimed rewards")
            # Check events if needed
            claim_events = staking.events.RewardsClaimed().process_receipt(receipt)
            if claim_events:
                print("RewardsClaimed event confirmed")
        else:
            print("Rewards claim transaction failed")
            
    except Exception as e:
        print(f"Error in claim_rewards: {str(e)}")

def unstake_tokens(amount_in_ether=None):
    try:
        # If no amount specified, withdraw entire balance
        if amount_in_ether is None:
            stake_info = staking.functions.stakes(staker_account.address).call()
            amount = stake_info[0]  # Full staked amount
            amount_in_ether = w3.from_wei(amount, 'ether')
        else:
            amount = w3.to_wei(amount_in_ether, 'ether')
        
        print("\nBefore unstaking:")
        print_balances(staker_account.address, "Staker")
        
        # Prepare withdraw transaction (changed from unstake to withdraw)
        withdraw_tx = staking.functions.withdraw(
            amount
        ).build_transaction({
            'from': staker_account.address,
            'nonce': w3.eth.get_transaction_count(staker_account.address),
            'gasPrice': w3.eth.gas_price
        })
        
        # Send withdraw transaction
        receipt = send_transaction(withdraw_tx, staker_key)
        if not receipt:
            return
        
        print("\nAfter unstaking:")
        print_balances(staker_account.address, "Staker")
        
        if receipt.status == 1:
            print(f"Successfully withdrawn {amount_in_ether} MTK")
            # Check events if needed
            withdraw_events = staking.events.Withdrawn().process_receipt(receipt)
            if withdraw_events:
                print("Withdrawn event confirmed")
        else:
            print("Withdrawal transaction failed")
            
    except Exception as e:
        print(f"Error in unstake_tokens: {str(e)}")

# Workflow demonstration
# First, ensure the staker has tokens
def transfer_tokens_to_staker(amount_in_ether):
    try:
        amount = w3.to_wei(amount_in_ether, 'ether')
        
        # Prepare transfer transaction
        transfer_tx = token.functions.transfer(
            staker_account.address,  # recipient
            amount
        ).build_transaction({
            'from': deployer_account.address,
            'nonce': w3.eth.get_transaction_count(deployer_account.address),
            'gasPrice': w3.eth.gas_price
        })
        
        # Send transfer transaction
        receipt = send_transaction(transfer_tx, deployer_key)
        
        if receipt and receipt.status == 1:
            print(f"Successfully transferred {amount_in_ether} MTK to staker")
            print_balances(staker_account.address, "Staker after transfer")
        else:
            print("Token transfer failed")
    except Exception as e:
        print(f"Error transferring tokens: {str(e)}")

# Workflow for staker
def staker_workflow():
    # 1. First, ensure staker has tokens
    transfer_tokens_to_staker(500)  # Transfer 500 MTK to staker
    
    # 2. Deposit rewards by deployer (if not already done)
    deposit_rewards(500)  # Deposit 500 MTK as rewards
    
    # 3. Staker stakes tokens
    stake_tokens(200)  # Stake 200 MTK
    
    # 4. Simulate some time passing (in a real scenario, this would be actual blockchain time)
    print("\nSimulating time passing to accumulate rewards...")
    
    # 5. Claim rewards
    claim_rewards()
    
    # 6. Unstake tokens
    unstake_tokens()  # Unstake entire balance


# Execute the staker workflow
staker_workflow()

Successfully transferred 500 MTK to staker

Staker after transfer Balances:
Token balance: 1602.74 MTK
Staked balance: 0 MTK

Before deposit:

Deployer Balances:
Token balance: 997000 MTK
Staked balance: 0 MTK
Reward pool in contract: 1297.26 MTK
Reward rate: 100000000000000
Contract token balance: 1297.26 MTK
Contract allowance from deployer: 0 MTK

After approval:
Reward pool in contract: 1297.26 MTK
Reward rate: 100000000000000
Contract token balance: 1297.26 MTK
Contract allowance from deployer: 500 MTK

After deposit:

Deployer Balances:
Token balance: 996500 MTK
Staked balance: 0 MTK
Reward pool in contract: 1797.26 MTK
Reward rate: 100000000000000
Contract token balance: 1797.26 MTK
Contract allowance from deployer: 0 MTK
Successfully deposited 500 MTK as rewards
RewardsDeposited event confirmed

Before staking:


/Users/qw246/miniforge3/envs/quant/lib/python3.10/site-packages/eth_utils/functional.py:47: UserWarning: The log with transaction hash: HexBytes('0x087eaf2a1e32755d967930ea08e1fe0df1c734100722f07687642d4f973b868e') and logIndex: 3 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  return callback(fn(*args, **kwargs))



Staker Balances:
Token balance: 1602.74 MTK
Staked balance: 0 MTK
Reward pool in contract: 1797.26 MTK
Reward rate: 100000000000000
Contract token balance: 1797.26 MTK
Contract allowance from deployer: 0 MTK

After staking:

Staker Balances:
Token balance: 1402.74 MTK
Staked balance: 200 MTK
Reward pool in contract: 1797.26 MTK
Reward rate: 100000000000000
Contract token balance: 1997.26 MTK
Contract allowance from deployer: 0 MTK
Successfully staked 200 MTK
Staked event confirmed

Simulating time passing to accumulate rewards...

Before claiming rewards:


/Users/qw246/miniforge3/envs/quant/lib/python3.10/site-packages/eth_utils/functional.py:47: UserWarning: The log with transaction hash: HexBytes('0xbbb1a73b87614c9eec726acbff40c1dd1cbef1159f8f0da7d78dbd92fb6acffa') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  return callback(fn(*args, **kwargs))



Staker Balances:
Token balance: 1402.74 MTK
Staked balance: 200 MTK

After claiming rewards:

Staker Balances:
Token balance: 1402.84 MTK
Staked balance: 200 MTK
Successfully claimed rewards
RewardsClaimed event confirmed


/Users/qw246/miniforge3/envs/quant/lib/python3.10/site-packages/eth_utils/functional.py:47: UserWarning: The log with transaction hash: HexBytes('0xc26dedec1d0e6463a51c701e8ef954e1413d64907b4d14bebad6d0eee24db21a') and logIndex: 2 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  return callback(fn(*args, **kwargs))



Before unstaking:

Staker Balances:
Token balance: 1402.84 MTK
Staked balance: 200 MTK

After unstaking:

Staker Balances:
Token balance: 1602.92 MTK
Staked balance: 0 MTK
Successfully withdrawn 200 MTK
Withdrawn event confirmed


/Users/qw246/miniforge3/envs/quant/lib/python3.10/site-packages/eth_utils/functional.py:47: UserWarning: The log with transaction hash: HexBytes('0x2eab3b71df025cd314b3dca66dc1ef7ddf6cfbf4a58c8e158256013d682495d7') and logIndex: 0 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  return callback(fn(*args, **kwargs))
/Users/qw246/miniforge3/envs/quant/lib/python3.10/site-packages/eth_utils/functional.py:47: UserWarning: The log with transaction hash: HexBytes('0x2eab3b71df025cd314b3dca66dc1ef7ddf6cfbf4a58c8e158256013d682495d7') and logIndex: 1 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  return callback(fn(*args, **kwargs))
/Users/qw246/miniforge3/envs/quant/lib/python3.10/site-packages/eth_utils/functional.py:47: UserWarning: The log with transaction hash: HexBytes('0x2eab3b71df025cd314b3dca66dc1ef